1. Import Packages

In [46]:
import pandas as pd
import numpy as np
import mysql.connector as db

import warnings
warnings.filterwarnings("ignore")

2. Read a CSV file

In [47]:
df = pd.read_csv("seccheck_data.csv")
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


3. Data Cleaning

i) Drop unwanted columns

In [48]:
df.drop(["driver_age_raw","violation_raw"],axis = 1,inplace = True)

ii) Identify missing values

In [49]:
df.isnull().sum()

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age                0
driver_race               0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64

In [50]:
df['search_type'].unique()

array(['Vehicle Search', 'Frisk', nan], dtype=object)

In [51]:
srhtype= df['search_type'].mode()[0]
srhtype

'Frisk'

In [52]:
df['search_type'].fillna(srhtype, inplace= True)

In [53]:
df.isnull().sum()

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age            0
driver_race           0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64

In [54]:
data1 = df.head(500)
data1

,stop_date,stop_time,country_name,driver_gender,driver_age,driver_race,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,19,Asian,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,58,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,76,Black,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,76,Black,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,75,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2020-01-01,8:15:00,USA,M,67,Black,Signal,False,Vehicle Search,Arrest,False,0-15 Min,False,DL37CK3012
496,2020-01-01,8:16:00,Canada,M,75,Asian,Other,True,Vehicle Search,Warning,False,16-30 Min,True,KA76OG9154
497,2020-01-01,8:17:00,USA,M,75,Hispanic,Speeding,True,Frisk,Arrest,False,30+ Min,False,TN10OV8122
498,2020-01-01,8:18:00,Canada,M,32,Hispanic,Other,False,Vehicle Search,Arrest,False,30+ Min,False,DL91QW3561


In [55]:
data1.dtypes

stop_date             object
stop_time             object
country_name          object
driver_gender         object
driver_age             int64
driver_race           object
violation             object
search_conducted        bool
search_type           object
stop_outcome          object
is_arrested             bool
stop_duration         object
drugs_related_stop      bool
vehicle_number        object
dtype: object

4. convert all rows into a list of tuples

In [56]:
value = data1.values
value

array([['2020-01-01', '0:00:00', 'Canada', ..., '16-30 Min', True,
        'UP76DY3473'],
       ['2020-01-01', '0:01:00', 'India', ..., '16-30 Min', True,
        'RJ83PZ4441'],
       ['2020-01-01', '0:02:00', 'USA', ..., '16-30 Min', True,
        'RJ32OM7264'],
       ...,
       ['2020-01-01', '8:17:00', 'USA', ..., '30+ Min', False,
        'TN10OV8122'],
       ['2020-01-01', '8:18:00', 'Canada', ..., '30+ Min', False,
        'DL91QW3561'],
       ['2020-01-01', '8:19:00', 'USA', ..., '30+ Min', True,
        'KA94DL5663']], dtype=object)

In [57]:
num_array = np.array(value)
val = list(map(tuple, num_array))

val

[('2020-01-01',
  '0:00:00',
  'Canada',
  'M',
  19,
  'Asian',
  'Speeding',
  True,
  'Vehicle Search',
  'Ticket',
  True,
  '16-30 Min',
  True,
  'UP76DY3473'),
 ('2020-01-01',
  '0:01:00',
  'India',
  'M',
  58,
  'Other',
  'Other',
  False,
  'Vehicle Search',
  'Arrest',
  True,
  '16-30 Min',
  True,
  'RJ83PZ4441'),
 ('2020-01-01',
  '0:02:00',
  'USA',
  'M',
  76,
  'Black',
  'Speeding',
  False,
  'Frisk',
  'Ticket',
  True,
  '16-30 Min',
  True,
  'RJ32OM7264'),
 ('2020-01-01',
  '0:03:00',
  'Canada',
  'M',
  76,
  'Black',
  'DUI',
  True,
  'Frisk',
  'Warning',
  False,
  '0-15 Min',
  True,
  'RJ76TI3807'),
 ('2020-01-01',
  '0:04:00',
  'Canada',
  'M',
  75,
  'Other',
  'Other',
  False,
  'Vehicle Search',
  'Arrest',
  True,
  '16-30 Min',
  False,
  'WB63BB8305'),
 ('2020-01-01',
  '0:05:00',
  'Canada',
  'F',
  73,
  'Other',
  'Other',
  True,
  'Vehicle Search',
  'Ticket',
  False,
  '16-30 Min',
  False,
  'MH89ZF7676'),
 ('2020-01-01',
  '0:06:00'

5. Database Connection

In [58]:
connection = db.connect(
    host = "localhost",
    user = "root",
    password = "sandy2505",
    database = "db1"
)
curr = connection.cursor()

In [61]:
q = """
create table digital_logs
(
    stop_date DATE,
    stop_time TIME,                
    country_name VARCHAR(50),         
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation VARCHAR(100),
    search_conducted VARCHAR(50),
    search_type VARCHAR(50),
    stop_outcome VARCHAR(50),
    is_arrested  VARCHAR(20),
    stop_duration VARCHAR(50),
    drugs_related_stop VARCHAR(50),
    vehicle_number VARCHAR(50) 
 )
"""
curr.execute(q)

In [62]:
insert = """
insert into digital_logs
values
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

curr.executemany(insert,val)
connection.commit()